In [1]:
%load_ext autoreload 
%autoreload 2

from preprocessing import FileIO, Vectorizor
from opensearch_interface import OpenSearchClient
from index_templates import youtube_body

import os
import time
from rich import print
from dotenv import load_dotenv
load_env=load_dotenv('./.env', override=True)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Instantiate OpenSearch Client

In [2]:
model_path = 'sentence-transformers/all-MiniLM-L6-v2'
model_path_on_disk = os.environ['ST_MODEL_PATH']

In [3]:
os_endpoint = os.environ['OPENSEARCH_ENDPOINT']
os_username = os.environ['OPENSEARCH_USERNAME']
os_password = os.environ['OPENSEARCH_PASSWORD']

In [4]:
osclient = OpenSearchClient(hosts=os_endpoint,http_auth=(os_username, os_password))

### Display Existing Indices

In [13]:
# osclient.indices.delete(index_name)
osclient.show_indexes()

health status index                     uuid                   pri rep docs.count docs.deleted store.size pri.store.size
green  open   .opensearch-observability SszFhNsdQn6XzZRbNH_maQ   1   0          0            0       208b           208b
yellow open   impact-theory-minilm-196  lsZrOS9WTCihEe1RUWoQeA   3   1      37007            0    369.1mb        369.1mb
green  open   .opendistro_security      cDQQKOHYRYu-Wrev6dWoAw   1   0         10            0     52.7kb         52.7kb
green  open   .kibana_1                 EMgTmu59TJSqO6po7c6RNg   1   0          1            0      5.1kb          5.1kb



### Load saved data from previous Ingest/Split/Vectorize process

In [6]:
data_path = './practice_data/impact_theory_minilm_196.parquet'

In [7]:
data = FileIO().load_parquet(data_path)

Shape of data: (37007, 17)
Memory Usage: 4.55+ MB


### Preview some data stats

In [8]:
data_keys = sorted(list(data[0].keys()))
emb_dim = len(data[0]['content_embedding'])
line_break = '-'*100
print(f'Data Keys: {data_keys}\n{line_break}\nEmbedding Dimension: {emb_dim}')

Data Keys: ['age_restricted', 'author', 'channel_id', 'content', 'content_embedding', 'description', 'doc_id', 
'episode_num', 'episode_url', 'keywords', 'length', 'playlist_id', 'publish_date', 'thumbnail_url', 'title', 
'video_id', 'views']
----------------------------------------------------------------------------------------------------
Embedding Dimension: 384

In [9]:
# [d for d in data if d['doc_id'] == 'mJNM7iLAibU_4']

### Data Indexing

In [10]:
index_name = 'impact-theory-minilm-196'

In [11]:
#Review Indexing Body
youtube_body

{'settings': {'number_of_shards': 3,
  'refresh_interval': '30s',
  'index': {'knn': False}},
 'mappings': {'properties': {'title': {'type': 'text', 'index': 'true'},
   'unique_id': {'type': 'keyword', 'index': 'false'},
   'group_id': {'type': 'short', 'index': 'false'},
   'video_id': {'type': 'keyword', 'index': 'false'},
   'playlist_id': {'type': 'keyword', 'index': 'false'},
   'episode_url': {'type': 'keyword', 'index': 'false'},
   'episode_num': {'type': 'short', 'index': 'false'},
   'description': {'type': 'text', 'index': 'true'},
   'length': {'type': 'long', 'index': 'false'},
   'publish_date': {'type': 'keyword', 'index': 'false'},
   'views': {'type': 'long', 'index': 'false'},
   'thumbnail_url': {'type': 'keyword', 'index': 'false'},
   'content': {'type': 'text', 'index': 'true'},
   'doc_id': {'type': 'keyword', 'index': 'false'}}}}

In [12]:
%%time
osclient.document_indexer(index_name=index_name, data=data, chunk_size=5, body_template=youtube_body, semantic_index=True)
time.sleep(30)

2023-11-02 21:24:05.235 | INFO     | opensearch_interface:document_indexer:212 - The ** impact-theory-minilm-196 ** index was created
2023-11-02 21:24:05.237 | INFO     | opensearch_interface:document_indexer:219 - The # of documents to be indexed = 37007
100%|███████████████████████████████████████████████████████████████████████| 37007/37007 [02:41<00:00, 229.18Docs Indexed/s]


CPU times: user 17.6 s, sys: 1.44 s, total: 19.1 s
Wall time: 3min 12s


In [62]:
osclient.indices.refresh(index_name)

{'_shards': {'total': 6, 'successful': 3, 'failed': 0}}

In [57]:
osclient.indices.delete(index_name)

{'acknowledged': True}

## Test Search on New Index

In [14]:
query = "Like why the new company, Cellularity, why are you doing that? It's gonna take an incredible amount of time, energy, and effort."

### Keyword Search

In [17]:
kw_response = osclient.keyword_search(query=query, index=index_name, size=5)
print(kw_response)

[
    {
        '_index': 'impact-theory-minilm-196',
        '_id': '6x_tkYsB1OxrodOh9b8k',
        '_score': 47.884064,
        '_source': {
            'episode_url': 'https://www.youtube.com/watch?v=mJNM7iLAibU&list=PL8qcvQ7Byc3OJ02hbWJbHWePh4XEg3cvo',
            'length': 2247,
            'episode_num': 61,
            'title': 'A Future Without Death | Bob Hariri on Impact Theory',
            'thumbnail_url': 'https://i.ytimg.com/vi/mJNM7iLAibU/hq720.jpg',
            'doc_id': 'mJNM7iLAibU_4',
            'publish_date': '02-27-2018',
            'content': "It's true. Which is pretty crazy, so I'm really amped up to get you on and give you a 
chance to talk about some of the things that I know you've got going on in your life. And I think the most fun 
place to start with you is you've been absurdly successful by every worldly measure, but you're still super driven.
What drives you? Like why the new company, Cellularity, why are you doing that? It's gonna take an incredible 
amount of time, energy, and effort. So what is the goal with the company and why are you so driven to make it 
happen? Well, you know, I think it goes back to one of our early conversations. When I remember flying into the 
little tiny airport near your offices outside of L.A., and I picked you up, remember? Yes.",
            'views': 30633,
            'video_id': 'mJNM7iLAibU'
        }
    },
    {
        '_index': 'impact-theory-minilm-196',
        '_id': '7B_tkYsB1OxrodOh9b8k',
        '_score': 25.552937,
        '_source': {
            'episode_url': 'https://www.youtube.com/watch?v=mJNM7iLAibU&list=PL8qcvQ7Byc3OJ02hbWJbHWePh4XEg3cvo',
            'length': 2247,
            'episode_num': 61,
            'title': 'A Future Without Death | Bob Hariri on Impact Theory',
            'thumbnail_url': 'https://i.ytimg.com/vi/mJNM7iLAibU/hq720.jpg',
            'doc_id': 'mJNM7iLAibU_5',
            'publish_date': '02-27-2018',
            'content': "Yes. And we started talking about where we thought cellular medicine and stem cells were 
gonna go to change the lives of people that we care about. And we talked about the fact that a new revolution in 
medicine was upon us. And we got very far along, but we haven't completed the journey yet. And so part of the 
driving force behind Cellularity and my efforts to lead us into this next evolution is to bring all of these 
therapeutic products to the mainstream. Just like the remarkable things you've done in nutrition would not have 
been possible if it was unapproachable economically, right? So that was part of this whole mission. So Cellularity 
is a product of 20 years of effort to turn living cells into medicines, identify the best places where they work, 
and then deliver them in a scalable economical way so that everybody has access. But why do you give a shit? It's 
gonna be so much work.",
            'views': 30633,
            'video_id': 'mJNM7iLAibU'
        }
    },
    {
        '_index': 'impact-theory-minilm-196',
        '_id': '2R_skYsB1OxrodOh0IMB',
        '_score': 22.049007,
        '_source': {
            'episode_url': 'https://www.youtube.com/watch?v=iWtRUN0SQhI&list=PL8qcvQ7Byc3OJ02hbWJbHWePh4XEg3cvo',
            'length': 9333,
            'episode_num': 293,
            'title': 'How To Create A MORNING ROUTINE To Achieve Your MOST AMBITIOUS GOALS In 2023 | Tom Bilyeu',
            'thumbnail_url': 'https://i.ytimg.com/vi/iWtRUN0SQhI/hq720.jpg',
            'doc_id': 'iWtRUN0SQhI_111',
            'publish_date': '11-24-2022',
            'content': "Right? Man, it was incredible. It was so beautiful. The look on their face at that moment 
where they see themselves for the first time in front of the mirror and they are seeing something totally new. 
They're seeing a version of themselves that they did not know was there. And the ability to give that gift, like it
is so easy to get lost in the day-to-day grind of what you do, what any of us do. But if at the core o

### Vector Search

In [19]:
vec_response = osclient.vector_search(query, index_name, size=5)
print(vec_response)

[
    {
        '_index': 'impact-theory-minilm-196',
        '_id': '8B_skYsB1OxrodOh2IZA',
        '_score': 0.56157386,
        '_source': {
            'episode_url': 'https://www.youtube.com/watch?v=OAXi4jcCrWA&list=PL8qcvQ7Byc3OJ02hbWJbHWePh4XEg3cvo',
            'length': 6451,
            'episode_num': 277,
            'title': '“The First Hour Of The Morning Is CRUCIAL” - Millionaire Habits For SUCCESS! | Bedros 
Keuilian',
            'thumbnail_url': 'https://i.ytimg.com/vi/OAXi4jcCrWA/hq720.jpg',
            'doc_id': 'OAXi4jcCrWA_5',
            'publish_date': '10-04-2022',
            'content': "but If you are inefficient to your point about okay, we all have the same number of hours 
So it comes down to how are you spending your time? But there's something like I find myself wanting to say more 
words because there's something ineffable that I'm not capturing yet which is there is a really subtle difference 
between a high-performance day and Mediocre day and when I think about certainly the thing that has been my 
superpower is I'm able to generate momentum So I can go from standing still nobody knows who I am nobody believes 
in what I'm trying to do We don't have a company incorporated like all of that and then oh, you know Those firms 
like getting the machine moving is so hard That's what I can do and whether that's a new project whether that's a 
whole company, whatever.",
            'views': 259625,
            'video_id': 'OAXi4jcCrWA'
        }
    },
    {
        '_index': 'impact-theory-minilm-196',
        '_id': '_h_ukYsB1OxrodOhN8vY',
        '_score': 0.5471567,
        '_source': {
            'episode_url': 'https://www.youtube.com/watch?v=aIVCyp_7Cqc&list=PL8qcvQ7Byc3OJ02hbWJbHWePh4XEg3cvo',
            'length': 2998,
            'episode_num': 19,
            'title': 'Billionaire REVEALS The Key Habits That Will CHANGE Your Life! | John Paul Dejoria',
            'thumbnail_url': 'https://i.ytimg.com/vi/aIVCyp_7Cqc/hq720.jpg',
            'doc_id': 'aIVCyp_7Cqc_64',
            'publish_date': '05-02-2017',
            'content': "The best way to explain it is by actually doing something. Let's take a huge business 
throughout the world, one of the biggest, telecommunications. You have giant phone companies, you have servers that
on your cellular phone, it's almost like a lifestyle now, right? Well, how does someone get in that industry and 
all of a sudden you got all the giants you're going up against that have multi millions and billions of dollars, 
how do you do it? Well, you look at what the industry is lacking and pull the industry into it. Give you an 
example, it's one of my latest companies, ROK, R-O-K Mobile. ROK Mobile, believe it or not, we found the openings, 
I'll tell you how we did it.",
            'views': 1318528,
            'video_id': 'aIVCyp_7Cqc'
        }
    },
    {
        '_index': 'impact-theory-minilm-196',
        '_id': 'Ax_tkYsB1OxrodOh9cB1',
        '_score': 0.5351826,
        '_source': {
            'episode_url': 'https://www.youtube.com/watch?v=mJNM7iLAibU&list=PL8qcvQ7Byc3OJ02hbWJbHWePh4XEg3cvo',
            'length': 2247,
            'episode_num': 61,
            'title': 'A Future Without Death | Bob Hariri on Impact Theory',
            'thumbnail_url': 'https://i.ytimg.com/vi/mJNM7iLAibU/hq720.jpg',
            'doc_id': 'mJNM7iLAibU_28',
            'publish_date': '02-27-2018',
            'content': "Not a lot of sycophants who just wanna sort of make you happy. And also, you know what? You
need to bring in new blood with completely opposing points of view. So we've tried to do that at Cellularity. And I
think it's very helpful. But at the same time, I want people to fully exploit the power of this connectivity to our
original parent company, to the companies we're strategically partnered with, to allied companies that are equity 
holders in Cellularity, but also business partners. And by doing that, I think we avoid lots of the problem

### Hybrid Search

In [20]:
hyb_response = osclient.hybrid_search(query, index_name, index_name, kw_size=3, vec_size=3)
print(hyb_response)

[
    {
        '_index': 'impact-theory-minilm-196',
        '_id': '6x_tkYsB1OxrodOh9b8k',
        '_score': 47.884064,
        '_source': {
            'episode_url': 'https://www.youtube.com/watch?v=mJNM7iLAibU&list=PL8qcvQ7Byc3OJ02hbWJbHWePh4XEg3cvo',
            'length': 2247,
            'episode_num': 61,
            'title': 'A Future Without Death | Bob Hariri on Impact Theory',
            'thumbnail_url': 'https://i.ytimg.com/vi/mJNM7iLAibU/hq720.jpg',
            'doc_id': 'mJNM7iLAibU_4',
            'publish_date': '02-27-2018',
            'content': "It's true. Which is pretty crazy, so I'm really amped up to get you on and give you a 
chance to talk about some of the things that I know you've got going on in your life. And I think the most fun 
place to start with you is you've been absurdly successful by every worldly measure, but you're still super driven.
What drives you? Like why the new company, Cellularity, why are you doing that? It's gonna take an incredible 
amount of time, energy, and effort. So what is the goal with the company and why are you so driven to make it 
happen? Well, you know, I think it goes back to one of our early conversations. When I remember flying into the 
little tiny airport near your offices outside of L.A., and I picked you up, remember? Yes.",
            'views': 30633,
            'video_id': 'mJNM7iLAibU'
        }
    },
    {
        '_index': 'impact-theory-minilm-196',
        '_id': '8B_skYsB1OxrodOh2IZA',
        '_score': 0.56157386,
        '_source': {
            'episode_url': 'https://www.youtube.com/watch?v=OAXi4jcCrWA&list=PL8qcvQ7Byc3OJ02hbWJbHWePh4XEg3cvo',
            'length': 6451,
            'episode_num': 277,
            'title': '“The First Hour Of The Morning Is CRUCIAL” - Millionaire Habits For SUCCESS! | Bedros 
Keuilian',
            'thumbnail_url': 'https://i.ytimg.com/vi/OAXi4jcCrWA/hq720.jpg',
            'doc_id': 'OAXi4jcCrWA_5',
            'publish_date': '10-04-2022',
            'content': "but If you are inefficient to your point about okay, we all have the same number of hours 
So it comes down to how are you spending your time? But there's something like I find myself wanting to say more 
words because there's something ineffable that I'm not capturing yet which is there is a really subtle difference 
between a high-performance day and Mediocre day and when I think about certainly the thing that has been my 
superpower is I'm able to generate momentum So I can go from standing still nobody knows who I am nobody believes 
in what I'm trying to do We don't have a company incorporated like all of that and then oh, you know Those firms 
like getting the machine moving is so hard That's what I can do and whether that's a new project whether that's a 
whole company, whatever.",
            'views': 259625,
            'video_id': 'OAXi4jcCrWA'
        }
    },
    {
        '_index': 'impact-theory-minilm-196',
        '_id': '7B_tkYsB1OxrodOh9b8k',
        '_score': 25.552937,
        '_source': {
            'episode_url': 'https://www.youtube.com/watch?v=mJNM7iLAibU&list=PL8qcvQ7Byc3OJ02hbWJbHWePh4XEg3cvo',
            'length': 2247,
            'episode_num': 61,
            'title': 'A Future Without Death | Bob Hariri on Impact Theory',
            'thumbnail_url': 'https://i.ytimg.com/vi/mJNM7iLAibU/hq720.jpg',
            'doc_id': 'mJNM7iLAibU_5',
            'publish_date': '02-27-2018',
            'content': "Yes. And we started talking about where we thought cellular medicine and stem cells were 
gonna go to change the lives of people that we care about. And we talked about the fact that a new revolution in 
medicine was upon us. And we got very far along, but we haven't completed the journey yet. And so part of the 
driving force behind Cellularity and my efforts to lead us into this next evolution is to bring all of these 
therapeutic products to the mainstream. Just like the remarkable things you've done in nutrition wo